1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다, 2. 문서를 쪼갠다

In [8]:
%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax_with_markdown.docx")
documents = loader.load_and_split(text_splitter=text_splitter)

In [27]:
len(documents)

225

# 3. 임베딩

In [14]:
%pip install -U langchain_openai

  Attempting uninstall: langchain_openai
    Found existing installation: langchain-openai 0.1.23
    Uninstalling langchain-openai-0.1.23:
      Successfully uninstalled langchain-openai-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.2.2 requires langchain-core<0.3,>=0.2.35, but you have langchain-core 0.3.0 which is incompatible.
langchain-upstage 0.2.2 requires langchain-openai==0.1.23, but you have langchain-openai 0.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [33]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [2]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [35]:
import os

from pinecone import Pinecone
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore

load_dotenv()

index_name= 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(documents, embedding, index_name=index_name)

# 4. 질문이 있을 때 벡터 데이터베이스에 유사도 검색

In [54]:
query = '연봉 5500만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, k=5)

In [45]:
retrieved_docs

[Document(id='b5e52dc7-8d81-47a8-875d-29147a97aa89', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

# 5. 유사도 검색으로 가져온 문서를 질문과 함께 LLM에게 전달

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [7]:
ai_response = llm.invoke(prompt)

In [8]:
ai_response.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 과세표준을 계산하고, 그 다음으로 기본세율을 적용해야 합니다. 여기서는 기본적인 계산 방식을 설명드리겠습니다.\n\n1. **총 급여에서 각종 공제 항목을 차감**:\n    - **근로소득공제**: 연봉 5천만원에 해당하는 근로소득공제 금액을 계산합니다.\n    - **인적공제 및 기타 공제**: 인적공제 (기본공제, 추가공제 등)와 기타 공제 항목 (보험료 공제, 의료비 공제, 교육비 공제 등)을 적용합니다.\n\n2. **과세표준 계산**:\n    - 총 급여액에서 근로소득공제와 인적공제 및 기타 공제를 차감하여 과세표준을 산출합니다.\n\n3. **기본세율 적용**:\n    - 과세표준에 기본세율을 적용하여 산출세액을 계산합니다.\n\n4. **세액공제 및 감면**:\n    - 산출세액에서 세액공제 및 감면 항목 (근로소득세액공제 등)을 차감하여 최종 산출세액을 계산합니다.\n\n**예시 계산**:\n\n1. **근로소득공제 계산**:\n    - 연봉 5천만원에 대한 근로소득공제는 다음과 같습니다:\n      ```\n      총급여액 5천만원에 대한 근로소득공제 = 1,200만원 + (총급여액 - 4,500만원) * 15%\n      = 1,200만원 + (5,000만원 - 4,500만원) * 15%\n      = 1,200만원 + 75만원\n      = 1,275만원\n      ```\n\n2. **과세표준 계산**:\n    - 연봉 5천만원에서 근로소득공제 1,275만원을 차감:\n      ```\n      총급여액 5천만원 - 근로소득공제 1,275만원 = 3,725만원\n      ```\n    - 인적공제 및 기타 공제 항목을 적용합니다. 여기서는 기본공제 (본인 150만원)만 적용한다고 가정:\n      ```\n      과세표준 = 3,725만원 - 기본공제 150만원 = 3,575만원\n      ```\n\n3. **기본세율 적용**:\n    -

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 기본세율은 과세표준에 따라 다르며, 다음과 같습니다:\n\n1. 과세표준 1,200만원 이하: 6%\n2. 과세표준 1,200만원 초과 4,600만원 이하: 15%\n3. 과세표준 4,600만원 초과 8,800만원 이하: 24%\n4. 과세표준 8,800만원 초과 1억5천만원 이하: 35%\n5. 과세표준 1억5천만원 초과 3억원 이하: 38%\n6. 과세표준 3억원 초과 5억원 이하: 40%\n7. 과세표준 5억원 초과: 42%\n\n연봉 5천만원의 경우, 과세표준은 다음과 같이 계산됩니다:\n\n1. 연봉 5천만원에서 근로소득공제와 각종 공제를 차감한 금액이 과세표준이 됩니다.\n2. 근로소득공제는 다음과 같습니다:\n   - 연봉 5천만원의 경우: 3,600만원 + (연봉 - 4,500만원) * 5% = 3,600만원 + (5,000만원 - 4,500만원) * 5% = 3,600만원 + 25만원 = 3,625만원\n\n따라서, 과세표준은 5천만원 - 3,625만원 = 1,375만원입니다.\n\n이제, 기본세율표를 적용하여 소득세를 계산합니다:\n\n- 과세표준 1,200만원 이하 부분: 1,200만원 * 6% = 72만원\n- 과세표준 1,200만원 초과 1,375만원 이하 부분: (1,375만원 - 1,200만원) * 15% = 175만원 * 15% = 26.25만원\n\n따라서, 총 소득세는 72만원 + 26.25만원 = 98.25만원입니다.\n\n연봉 5천만원인 직장인의 소득세는 대략 98.25만원이 됩니다. 다만, 이 계산은 근로소득공제와 기본세율만을 고려한 것이며, 실제 소득세는 각종 세액공제, 보험료 공제 등 다른 공제를 고려해야 합니다. 정확한 금액은 국세청 홈택스 등의 소득세 계산기를 이용해 보시는 것이 좋습니다.'

# 5-1. 직접 프롬프트를 만들지 말고, RetrievalQA, langchainhub 이용해서 간편하게 질문과 문서 던지기

In [20]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [46]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(search_kwargs={'k': 3}),
    chain_type_kwargs={"prompt": prompt}
)

In [47]:
ai_message = qa_chain({"query": query})

In [48]:
ai_message

{'query': '연봉 5500만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5,500만원인 거주자의 소득세는 624만원 + (5,500만원 - 5,000만원) * 24% = 744만원입니다.'}

# 만약, 질문에 직장인이 들어오면 거주자로 변경한다.

- 그 방법은 LCEL(LangChain Expression Language)를 사용해서, 어떤 작업을 수행하고 곧바로 또 다른 작업을 수행하게 한다.
- ChatPromptTemplate을 사용해서 프롬프트를 만든다.
- 여기서 우리가 만든 dictionary를 주고 질문이 이 dictionary에 해당한다면 단어를 변경해달라고 요청하는 프롬프트를 만든다.
- 그리고 질문(question)은 변수로 받는다.
- 그래서 프롬프트를 먼저 실행하고, llm 모델에 프롬프트를 던지고, 결과를 아웃풋으로 출력하는 3가지 작업을 LCEL로 진행하는 chain을 dictionary_chain으로 만든다.
- tax_chain이라는 chain을 만들고, dictionary_chain을 통해 나온 아웃풋을 "query"에 넣는다.
- 그 값을 qa_chain에 전달한다.
- 최종적으로 실행은 tax_chain.invoke({"question": query})로 실행하면 된다. 왜냐하면 dictionary_chain이 "question"이라는 변수를 받으니까

In [56]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약, 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [57]:
new_question = dictionary_chain.invoke({"question": query})

In [55]:
query

'연봉 5500만원인 직장인의 소득세는 얼마인가요?'

In [58]:
new_question

'    연봉 5500만원인 거주자의 소득세는 얼마인가요?'

In [59]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [60]:
ai_response =tax_chain.invoke({"question": query})

In [61]:
ai_response

{'query': '변경된 질문: 연봉 5500만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5500만원인 거주자의 소득세는 624만원 + (5500만원 - 5000만원) * 24% = 744만원입니다.'}